In [1]:
# импортируем необходимые библиотеки
import pandas as pd 
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import GridSearchCV
import numpy as np

In [2]:
# загружаем исходные данные из файла Excel с помощью метода read_excel библиотеки Pandas
df = pd.read_excel('ds_teachers.xlsx')

In [3]:
# удаляем дубликаты строк
df.drop_duplicates(inplace=True)
# удаляем строки, содержащие пропущенные значения
df.dropna(inplace=True)
# удаляем строки, содержащие определенные значения в столбце 'value_shortTitle'
df.drop(df[df['value_shortTitle'] == 'недсд.'].index, inplace = True)
df.drop(df[df['value_shortTitle'] == 'уваж.'].index, inplace = True)
df.drop(df[df['value_shortTitle'] == 'недоп.'].index, inplace = True)

# заменяем значения в столбце 'value_shortTitle' на числовые значения
No_val = [
    ['з', 5],
    ['н/я', 0],
    ['н/з', 2]
]
for i in No_val:
    df['value_shortTitle'] = df['value_shortTitle'].replace(i[0], i[1])


In [4]:
# создаем новый DataFrame, копируя исходный
df_dup = df

In [5]:
# выбираем только нужные столбцы из DataFrame и сохраняем в новый DataFrame
teacher_short_df = df[['teacher','student_booknumber','value_shortTitle']]

In [6]:
# удаляем строки, содержащие точку и запятую в столбце 'teacher'
teacher_short_df.drop(teacher_short_df[teacher_short_df['teacher'].str.contains('.,')].index, inplace = True)

/tmp/ipykernel_48778/2999483786.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  teacher_short_df.drop(teacher_short_df[teacher_short_df['teacher'].str.contains('.,')].index, inplace = True)


In [7]:

# выводим количество уникальных значений в каждом столбце нового DataFrame
teacher_short_df.nunique()

teacher               481
student_booknumber    999
value_shortTitle        5
dtype: int64

In [8]:
# преобразуем значения в столбце 'value_shortTitle' в целочисленный тип
teacher_short_df['value_shortTitle'].astype(int)

0        5
1        4
2        5
3        4
4        5
        ..
79808    5
79809    3
79810    3
79811    3
79812    3
Name: value_shortTitle, Length: 69109, dtype: int64

In [9]:

# получаем список преподавателей
teacher_list = teacher_short_df['teacher'].unique()

In [10]:

# создаем пустой список arr для хранения результатов
arr = []

In [11]:
# обходим список преподавателей
for number_iter,cnt in enumerate(teacher_list):
    # фильтруем DataFrame по преподавателю cnt
    temp_df = teacher_short_df[teacher_short_df['teacher'] == cnt]
    # получаем количество уникальных студентов, оценок и среднюю оценку по преподавателю cnt
    dep_count_stud = temp_df['student_booknumber'].nunique()
    dep_count_marks = sum(temp_df['value_shortTitle'])
    dep_avg = dep_count_marks / dep_count_stud
    # добавляем запись со значениями в список arr
    arr.append([cnt,number_iter,dep_count_stud,dep_avg])


In [12]:
# задаем названия столбцов для нового DataFrame
colname = ['teacher', 'cat_teacher','cnt_stud', 'avg_mark']
# создаем новый DataFrame с помощью списка arr и названий столбцов colname
df_cnt = pd.DataFrame(arr, columns= colname)

In [13]:
df_cnt

,teacher,cat_teacher,cnt_stud,avg_mark
0,Соловьев С.В.,0,44,20.181818
1,Катункина Е.В.,1,48,24.312500
2,Тепляков А.Г.,2,168,5.720238
3,Султанова Е.С.,3,20,6.100000
4,Романова Н.Г.,4,227,8.916300
...,...,...,...,...
476,Гумерова А.А.,476,18,4.111111
477,Лукашенко О.А.,477,7,10.000000
478,Чельцов М.В.,478,10,8.500000
479,Басманова Л.В.,479,5,10.000000


In [32]:
# сливаем два DataFrame по столбцу 'teacher'
df_feat = pd.merge(teacher_short_df, df_cnt[['teacher', 'cat_teacher','cnt_stud', 'avg_mark']], on='teacher')

In [33]:
df_feat

,teacher,student_booknumber,value_shortTitle,cat_teacher,cnt_stud,avg_mark
0,Соловьев С.В.,171158,5,0,44,20.181818
1,Соловьев С.В.,171158,4,0,44,20.181818
2,Соловьев С.В.,171158,5,0,44,20.181818
3,Соловьев С.В.,191629,5,0,44,20.181818
4,Соловьев С.В.,191630,5,0,44,20.181818
...,...,...,...,...,...,...
69104,Басманова Л.В.,150692,5,479,5,10.000000
69105,Басманова Л.В.,194623,5,479,5,10.000000
69106,Басманова Л.В.,181989,5,479,5,10.000000
69107,Гинтофт А.С.,180149,3,480,2,1.500000


In [34]:
# создаем объект OrdinalEncoder для преобразования категориальных значений в числовые
oe = OrdinalEncoder()

# преобразуем категории в числовые значения
encoded_cat_teacher = oe.fit_transform(df_feat[['cat_teacher']])

# заменяем значения в столбце 'teacher' на закодированные значения
df_feat['teacher'] = encoded_cat_teacher.flatten()

In [17]:
df_feat

,teacher,student_booknumber,value_shortTitle,cat_teacher,cnt_stud,avg_mark
0,0.0,171158,5,0,44,20.181818
1,0.0,171158,4,0,44,20.181818
2,0.0,171158,5,0,44,20.181818
3,0.0,191629,5,0,44,20.181818
4,0.0,191630,5,0,44,20.181818
...,...,...,...,...,...,...
69104,479.0,150692,5,479,5,10.000000
69105,479.0,194623,5,479,5,10.000000
69106,479.0,181989,5,479,5,10.000000
69107,480.0,180149,3,480,2,1.500000


In [18]:
# Разделяем данные на признаки (X) и целевую переменную (y)
X = df_feat.drop('avg_mark', axis=1)
y = df_feat['avg_mark']

# Разделяем данные на тренировочную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [19]:
#Создаем объект линейной регрессии и обучаем его на разделенных выборках
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [20]:
#Запускаем прогнозирование по модели
y_pred = lr.predict(X_test)

In [21]:
#Оцениваем качество модели на тестовых данных, рассчитывая среднеквадратичную ошибку (mean squared error, MSE):
mse = mean_squared_error(y_test, y_pred)
print('MSE: ', mse)
#Поскольку MSE показывает слишком высокие значения, необходимо подбирать гиперпараметры

MSE:  85.07015332432233


In [22]:
#Создаем сетку параметров
param_grid = {
    'fit_intercept': [True, False],
    'copy_X': [True, False],
    'n_jobs': [1, -1],
    'positive': [True, False]
}


# Создаем объект GridSearchCV и передаем ему модель, сетку параметров и количество фолдов для кросс-валидации
grid_search = GridSearchCV(lr, param_grid, cv=5)

# Обучаем объект GridSearchCV на тренировочных данных
grid_search.fit(X_train, y_train)

# Получаем лучшую комбинацию параметров и соответствующее значение MSE
print('Best parameters: ', grid_search.best_params_)
print('MSE: ', grid_search.best_score_)

Best parameters:  {'copy_X': True, 'fit_intercept': True, 'n_jobs': 1, 'positive': False}
MSE:  0.2108463934848035


In [23]:
#Создаем новую модель линейной регрессии по подобранным параметрам
lr_new = LinearRegression(copy_X=True, fit_intercept=True, n_jobs= 1, positive=False)

In [24]:
#Проводим обучение, тестирование модели по новым гиперпараметрам
lr_new.fit(X_train, y_train)
y_pred_new = lr_new.predict(X_test)
mse_new = mean_squared_error(y_test, y_pred_new)
print('MSE for new model: ', mse_new)
#Значение MSE слишком высокое - отправляем на перемоделирование

MSE for new model:  85.07015332432233


In [25]:
y_pred = grid_search.best_estimator_.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print('MSE on test set:', mse)

MSE on test set: 85.07015332432233


In [26]:


# Создаем объект модели
rf = RandomForestRegressor(n_estimators=100, random_state=42)

# Обучаем модель на тренировочных данных
rf.fit(X_train, y_train)

# Оцениваем качество модели на тестовых данных
y_pred = rf.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print('MSE on test set:', mse)
#MSE - тест показывает очень хорошие результаты

MSE on test set: 0.038249582366955776


In [27]:
#тем создается новый столбец "predicted_avg_mark" в датафрейме X_test с предсказанными средними оценками:
X_test['predicted_avg_mark'] = rf.predict(X_test)
print(X_test[['cat_teacher', 'cnt_stud', 'predicted_avg_mark']])

       cat_teacher  cnt_stud  predicted_avg_mark
110              0        44           20.181818
30717          142       174           10.362069
13889           44       211            6.696682
8317            15       254            9.637795
22914           96       189           17.957672
...            ...       ...                 ...
53614          297        78           13.846154
20941           79       145            6.937931
30380          140        46           22.956522
66011          416       115           11.652174
18937           73       420            5.135714

[13822 rows x 3 columns]


In [35]:
# Добавляем столбец с предсказанными средними оценками в DataFrame df_cnt
df_feat['predicted_avg_mark'] = rf.predict(df_feat.drop('avg_mark', axis=1))

# Рассчитываем взвешенную сумму средних оценок для каждого преподавателя
weighted_avg = []
for index, row in df_feat.iterrows():
    weighted_avg.append(row['predicted_avg_mark']*row['cnt_stud'])
df_feat['weighted_avg'] = weighted_avg
df_feat = df_feat.sort_values('weighted_avg', ascending=False)

# Выводим отсортированный по рейтингу DataFrame
print(df_feat[['teacher', 'weighted_avg']])

       teacher  weighted_avg
4765       5.0      19342.00
3809       5.0      19342.00
3785       5.0      19342.00
3784       5.0      19342.00
3783       5.0      19342.00
...        ...           ...
69107    480.0          5.16
69108    480.0          5.01
45945    241.0          3.87
45944    241.0          3.87
45943    241.0          3.87

[69109 rows x 2 columns]
